1. Introduction
=========
The original task was to generate the SDP relaxations of arbitrarily complex polynomial optimization problems of noncommuting variables, that is, the most general form of the NPA hierarchy [1] -- the tool for this conversion is called [Ncpol2sdpa](http://peterwittek.github.io/ncpol2sdpa/) [2]. Development started in MATLAB, with an eye on the libraries that achieve the same purpose for the Lasserre hierarchy, [Gloptipoly](http://homepages.laas.fr/henrion/software/gloptipoly/) and [SparsePOP](http://sparsepop.sourceforge.net/). Hence the approach was more algebraic from the beginning, rather than directly manipulating SDPs with [CVX](http://cvxr.com/cvx/), [SeDuMi](http://sedumi.ie.lehigh.edu/), or [Yalmip](http://users.isy.liu.se/johanl/yalmip/). Thus first it can be counterintuitive to use the library for quantum correlations. To further complicate things, MATLAB's noncommutative symbolic algebra capabilities were insufficient, hence development switched to using Python. The purpose of this notebook is to give a taste how it works.

2. Preliminiaries
=========
We found that the most difficult part is getting Python running with the necessary libraries. By far the easiest solution is to download [Anaconda](https://store.continuum.io/cshop/anaconda/), which bundles all necessary libraries and tools needed on Windows, Linux, and OS X. This is especially preferred if you are using a restricted computer where you do not have admin rights. You can then install the current stable version of Ncpol2sdpa by typing ``pip install ncpol2sdpa`` on the command line. Other relevant libraries could prove more difficult to install, but we won't detail that here. If all works fine, you should be able to import all functions from the module:

In [1]:
from ncpol2sdpa import *

Other than a working Python installation with the necessary modules, you will also need an SDP solver. [SDPA](http://sdpa.sourceforge.net/) is the most recommended solver, which can be infinitely painful to set up. The rest of the tutorial assumes that the SDPA executable is in the path. See the section on alternative solvers if you cannot get SDPA working. CVXOPT, for instance, is included in Anaconda, and should work out of the box.

3. Working with projectors
================
3.1 Tsirelson bound for CHSH at level 4
----------------------------------------------------
We take a simple CHSH scenario in the probability picture first. As a simple example, we want to find the Tsirelson bound of the inequality. Say, we want a result at level 4, then we need to specify the measurement configuration of Alice and Bob, and the inequality in the Collins-Gisin notation:

In [2]:
level = 4
Alice = [2, 2]
Bob = [2, 2]
I = [[ 0,   -1,    0 ],
     [-1,    1,    1 ],
     [ 0,    1,   -1 ]]
print(maximum_violation(Alice, Bob, I, level))

(-0.20710650757940174, -0.2071068414700042)


The value is negative because any kind of SDP generated **must** be a minimization, hence we flipped the sign of the objective function. The function ``maximum_violation`` hides the gory details. If we want to do something less trivial, we need to understand more of how it works. So let us do the same thing as above, but rolling out the steps. First of all, we need the symbolic operators representing the measurements of Alice and Bob:

In [3]:
A = generate_measurements(Alice, 'A')
B = generate_measurements(Bob, 'B')

For instance, now Alice has two sets of Hermitian operators:

In [4]:
A

[[A0], [A1]]

The first index encodes the measurement, the second the operator within the measurement. Clearly, $A_{01}$ and $A_{11}$ are missing, because we are in the Collins-Gisin notation.

These operators are Hermitian, but there is nothing telling that they are projectors. There algebra is defined through substitution rules that ensures a unique monomial for arbitrary multiplications of the operators of Alice and Bob.

In [5]:
substitutions = projective_measurement_constraints(A, B)

The above function can extend to arbitrary number of parties, measurements and measurement outputs. Let's take a look at the obtained substitutions:

In [6]:
substitutions

{B1*A1: A1*B1,
 B0*A0: A0*B0,
 B1**2: B1,
 B0**2: B0,
 A0**2: A0,
 B0*A1: A1*B0,
 B1*A0: A0*B1,
 A1**2: A1}

All we have are idempotency rules and commutations across the parties. If we had more then two outputs in a measurement, we would also see orthogonality rules. The only thing remaining before getting the relaxation is defining the objective function with the $I$ matrix, for which there is a helper function:

In [7]:
objective = define_objective_with_I(I, A, B)
objective

A0 - A0*B0 - A0*B1 - A1*B0 + A1*B1 + B0

Now we first initialize the SDP relaxation object. This step does not do any calculation, but it can take tons of parameters to fine-tune the resulting relaxation.

In [8]:
sdpRelaxation = SdpRelaxation(flatten([A, B]))

We are ready to generate the relaxation given the algebraic constraints. This is the step that is heavy in symbolic calculations. For complex problems or very high levels of relaxations, this can take a very long time.

In [9]:
sdpRelaxation.get_relaxation(level, objective=objective,
                             substitutions=substitutions)

Having obtained the relaxation, we can solve it with SDPA. Until now, all steps were symbolic. From here, everything is numeric.

In [10]:
sdpRelaxation.solve()
print(sdpRelaxation.primal, sdpRelaxation.dual)

(-0.20710650757940174, -0.2071068414700042)


3.2 Quantum bound on CGLMP at level 1+AB
-------------------------------------------------
We can quickly put this together from the above, while also preparing an extra set of monomials to be added to the moment matrix:

In [11]:
level = 1
Alice, Bob = [3, 3], [3, 3]
I = [[ 0, -1, -1,  0,  0],
     [-1,  1,  1,  0,  1],
     [-1,  1,  0,  1,  1],
     [ 0,  0,  1,  0, -1],
     [ 0,  1,  1, -1, -1]]
A = generate_measurements(Alice, 'A')
B = generate_measurements(Bob, 'B')
AB = [Ai*Bj for Ai in flatten(A) for Bj in flatten(B)]
substitutions = projective_measurement_constraints(A, B)
objective = define_objective_with_I(I, A, B)
sdpRelaxation = SdpRelaxation(flatten([A, B]))
sdpRelaxation.get_relaxation(level, objective=objective, substitutions=substitutions,
                             extramonomials=AB)
sdpRelaxation.solve()
print(sdpRelaxation.primal, sdpRelaxation.dual)

(-0.3157779808136485, -0.3157781081349144)


4. Moroder hierarchy and PPT conditions
=======================
Continuing with the CGLMP example, we can calculate the Moroder hierarchy and impose the PPT condition with changing only the SdpRelaxation object. Since the moment matrix has a tensor product structure, we need to supply two flat lists of operators instead of one, and ask for the Moroder structure with the PPT condition imposed. Everything else is identical, but we change the level to 2:

In [12]:
sdpRelaxation = MoroderHierarchy([flatten(A), flatten(B)], ppt=True)
level = 2
sdpRelaxation.get_relaxation(level, objective=objective, substitutions=substitutions)
sdpRelaxation.solve()
print(sdpRelaxation.primal, sdpRelaxation.dual)

(-0.03036149264516852, -0.03036223458167607)


Not surprisingly, the value is extremely close to zero.

5. Full probabilities
============
5.1 Local randomness
---------------------------
This is a bit more involved than the previous scenarios, but it should not be surprising, as we need a lot more constraints, which correspond to the observed correlations. We will use [QuTiP](http://qutip.org/) to calculate the observed correlations, but they could easily be computed by NumPy if QuTiP fails to install -- NumPy is included in Anaconda. We define the joint probability function for a CHSH scenario as follows

In [13]:
from math import sqrt, sin, cos, pi, atan
from qutip import tensor, basis, sigmax, sigmay, expect, qeye
def joint_probabilities():
    psi = (tensor(basis(2,0),basis(2,0)) + 
           tensor(basis(2,1),basis(2,1))).unit()
    A_0 = sigmax()
    A_1 = sigmay()
    B_0 = (-sigmay()+sigmax())/sqrt(2)
    B_1 = (sigmay()+sigmax())/sqrt(2)
    A_00 = (qeye(2) + A_0)/2
    A_10 = (qeye(2) + A_1)/2
    B_00 = (qeye(2) + B_0)/2
    B_10 = (qeye(2) + B_1)/2

    p=[expect(tensor(A_00, qeye(2)), psi),
       expect(tensor(A_10, qeye(2)), psi),
       expect(tensor(qeye(2), B_00), psi),
       expect(tensor(qeye(2), B_10), psi),
       expect(tensor(A_00, B_00), psi),
       expect(tensor(A_00, B_10), psi),
       expect(tensor(A_10, B_00), psi),
       expect(tensor(A_10, B_10), psi)]
    return p

Just as in the cases before, we need the operators for Alice and Bob. We need several of them, though, corresponding to the possible behaviour from which we can create the maximum guessing probability. Say, we are interested in the local guessing probability of Bob's second measurement, then we need two behaviours corresponding to the two projectors.

In [14]:
Alice, Bob = [2, 2], [2, 2]
P_0_A = generate_measurements(Alice, 'P_0_A')
P_0_B = generate_measurements(Bob, 'P_0_B')
P_1_A = generate_measurements(Alice, 'P_1_A')
P_1_B = generate_measurements(Bob, 'P_1_B')
substitutions = projective_measurement_constraints(P_0_A, P_0_B)
substitutions.update(projective_measurement_constraints(P_1_A, P_1_B))

As in the Moroder hierarchy, we generate the relaxation from separate lists of variables, but now to have a block-diagonal structure. Note that the moment matrices should not be normalized.

In [15]:
sdpRelaxation = SdpRelaxation([flatten([P_0_A, P_0_B]), flatten([P_1_A, P_1_B])],
                              normalized=False)

Next, we impose the constraints. The constraints are such that the relaxation should not generate localizing matrices. Such inequalities are called bounds in Ncpol2sdpa. Inequalities are always considered in >=0 format, so we only need to add the left-hand side.

In [16]:
probabilities = joint_probabilities()
moments = []
k=0
for i in range(len(Alice)):
    moments.append(P_0_A[i][0] + P_1_A[i][0] - probabilities[k])
    k += 1
for i in range(len(Bob)):
    moments.append(P_0_B[i][0] + P_1_B[i][0] - probabilities[k])
    k += 1
for i in range(len(Alice)):
    for j in range(len(Bob)):
        moments.append(P_0_A[i][0]*P_0_B[j][0] + P_1_A[i][0]*P_1_B[j][0] -
                          probabilities[k])
        k += 1

We also need to normalize the top-left element of the moment matrices to add up to one. For this, there is a special string-based syntax, which allows us to reference elements of the moment matrices yet to be generated:

In [17]:
moments.append("+0[0,0]+1[0,0]-1")

The last element of defining the problem is the objective function. Since we must include the second projector of Bob's second measurement, but it is abscent in the Collins-Gisin notation, we could produce it as $\mathbb{1}-B_{10}$. Since, however, the corresponding moment matrix is not normalized, we actually have to include the top-left element of that moment matrix instead of $\mathbb{1}$. For this, we have a similar syntax that allows us to add variables of the moment matrix to the objective function. Remember also that we must have minimization, hence all signs are inverted.

In [18]:
guessing_probability = - (P_0_B[1][0] - P_1_B[1][0])
extraterm="-1[0,0]"

Now we have everything to generate the SDP and solve the problem:

In [19]:
level = 2
sdpRelaxation.get_relaxation(level, objective=guessing_probability,
                             momentequalities=moments, substitutions=substitutions,
                             extraobjexpr=extraterm)
sdpRelaxation.solve()
print(sdpRelaxation.primal, sdpRelaxation.dual)

(-0.5006270052134052, -0.500627005212209)


5.2 Global randomness
----------------------------
The procedure is largely the same as in the case of local randomness, but the number of behaviours will be much larger.

In [20]:
P_00_A = generate_measurements(Alice, 'P_00_A')
P_00_B = generate_measurements(Bob, 'P_00_B')
P_01_A = generate_measurements(Alice, 'P_01_A')
P_01_B = generate_measurements(Bob, 'P_01_B')
P_10_A = generate_measurements(Alice, 'P_10_A')
P_10_B = generate_measurements(Bob, 'P_10_B')
P_11_A = generate_measurements(Alice, 'P_11_A')
P_11_B = generate_measurements(Bob, 'P_11_B')
substitutions = projective_measurement_constraints(P_00_A, P_00_B)
substitutions.update(projective_measurement_constraints(P_01_A, P_01_B))
substitutions.update(projective_measurement_constraints(P_10_A, P_10_B))
substitutions.update(projective_measurement_constraints(P_11_A, P_11_B))

Say, we are interested in the $x=0$, $y=0$ setting, then the guessing probability will be

In [21]:
guessing_probability = - (P_00_A[0][0]*P_00_B[0][0] +
                          P_01_A[0][0]*(1-P_01_B[0][0]) +
                          (1-P_10_A[0][0])*P_10_B[0][0] +
                          (1-P_11_A[0][0])*(1-P_11_B[0][0])) + 1.0
extraterm="-3[0,0]"

Bounds are defined identically:

In [22]:
moments = []
m = 0
for i in range(len(Alice)):
    for k in range(Alice[i]-1):
        moments.append(-P_00_A[i][k] - P_01_A[i][k] - P_10_A[i][k] - P_11_A[i][k] + probabilities[m])
        m += 1
for j in range(len(Bob)):
    for l in range(Bob[j]-1):
        moments.append(-P_00_B[j][l] - P_01_B[j][l] -P_10_B[j][l] - P_11_B[j][l] + probabilities[m])
        m += 1
for i in range(len(Alice)):
    for k in range(Alice[i]-1):
        for j in range(len(Bob)):
            for l in range(Bob[j]-1):
                moments.append(-P_00_A[i][k]*P_00_B[j][l]
                              -P_01_A[i][k]*P_01_B[j][l]
                              -P_10_A[i][k]*P_10_B[j][l]
                              -P_11_A[i][k]*P_11_B[j][l]
                              +probabilities[m])
                m += 1
moments.append("-0[0,0]-1[0,0]-2[0,0]-3[0,0]+1")

The solution is given by

In [23]:
sdpRelaxation = SdpRelaxation([flatten([P_00_A, P_00_B]),
                               flatten([P_01_A, P_01_B]),
                               flatten([P_10_A, P_10_B]),
                               flatten([P_11_A, P_11_B])],
                               normalized=False)
sdpRelaxation.get_relaxation(level, objective=guessing_probability,
                             momentequalities=moments, substitutions=substitutions,
                             extraobjexpr=extraterm)
sdpRelaxation.solve()
print(sdpRelaxation.primal, sdpRelaxation.dual)

(-0.4273418008417784, -0.42733142104509625)


6. Additional manipulation of the generated SDPs
============================
Until now, we never actually touched the generated SDPs, we just solved them by SDPA. We can, however, do arbitrary manipulations to them if we want to. In MATLAB, Yalmip provides a nice, high-level interface to work with conic optimization problems. The Python equivalent is [PICOS](http://picos.zib.de/). PICOS can be installed by typing ``pip install picos`` on the command line.

Once we generate an SDP relaxation, we can convert it to a PICOS problem. Let us, for instance, revisit the Moroder hierarchy for CGLMP, but ask for a non-normalized version.

In [24]:
level = 1
Alice, Bob = [3, 3], [3, 3]
I = [[ 0, -1, -1,  0,  0],
     [-1,  1,  1,  0,  1],
     [-1,  1,  0,  1,  1],
     [ 0,  0,  1,  0, -1],
     [ 0,  1,  1, -1, -1]]
A = generate_measurements(Alice, 'A')
B = generate_measurements(Bob, 'B')
substitutions = projective_measurement_constraints(A, B)
objective = define_objective_with_I(I, A, B)
sdpRelaxation = MoroderHierarchy([flatten(A), flatten(B)], ppt=True, normalized=False)
sdpRelaxation.get_relaxation(level, objective=objective, substitutions=substitutions)

We can convert this relaxation to a PICOS:

In [25]:
P = sdpRelaxation.convert_to_picos()
print(P)

---------------------
optimization problem  (SDP):
325 variables, 0 affine constraints, 325 vars in 1 SD cones

X 	: (25, 25), symmetric

	minimize X
such that
  X ≽ |0|
---------------------


Then we can manually normalized the moment matrix:

In [26]:
X = P.get_variable('X')
P.add_constraint(X[0, 0] == 1)

Just like Yalmip, PICOS can call a number of different solvers to deal with an SDP. By default, it looks for the most efficient one, and if it cannot find anything else, it will use [CVXOPT](http://cvxopt.org/). Here we explicitly force it use CVXOPT:

In [27]:
P.solve(solver='cvxopt')

{'cvxopt_sol': {'dual infeasibility': 1.2275769451556526e-09,
  'dual objective': 0.0549811036763854,
  'dual slack': 2.5804431425497183e-11,
  'gap': 6.591149605133918e-09,
  'iterations': 12,
  'primal infeasibility': 2.026546493978188e-09,
  'primal objective': 0.05498110365333003,
  'primal slack': 2.6195393825613468e-11,
  'relative gap': 1.1988027093688267e-07,
  'residual as dual infeasibility certificate': None,
  'residual as primal infeasibility certificate': None,
  's': <625x1 matrix, tc='d'>,
  'status': 'optimal',
  'x': <326x1 matrix, tc='d'>,
  'y': <81x1 matrix, tc='d'>,
  'z': <625x1 matrix, tc='d'>},
 'obj': 0.054981103664857714,
 'status': 'optimal',
 'time': 0.28634095191955566}

PICOS cannot use SDPA to solve a problem, but it can export to SDPA format.

7. Alternative SDP solvers
===============
7.1 Other members of the SDPA family
------------------------------------------------
The regular (double) precision solver of SDPA is the most difficult to compile. The source files are also available at the same place for the quad-double precision solver SDPA_QD, and the arbitrary-precision solver SDPA_GMP, which are infinitely easier to compile, requiring just one external library each.

The SDPA solvers come with a param.sdpa file for default parameter settings. This file is very similar for SDPA_QD and SDPA_GMP, but it is different for the double-precision solver. The parameter file should not be mixed up, as it leads to undefined behaviour. For SDPA_QD and SDPA_GMP, I recommend the following pattern of use:

1. Use the ``write_to_sdpa`` function of Ncpol2sdpa to write the SDP relaxation to a file that any SDP solver can use. The target directory should be where the param.sdpa file of SDPA_QD and SDPA_GMP resides. The filename should have the extension ``.dat-s``.
2. Run either SDPA_QD or SDPA_GMP in that directory.
3. Process the result in Ncpol2sdpa with ``read_sdpa_out`` if you want to analyze the solution matrix, find a rank loop (``find_rank_loop``), or do an SOS decomposition (``sos_decomposition``).

It is a bit tedious, but since the high-precision solvers are approximately 1,000x slower, this procedure ensures a fool-proof workflow.

7.2 Mosek and CVXOPT
---------------------------
For CVXOPT, the only option is via PICOS. It is, however, extremely slow. [Mosek](http://mosek.com/) is a better alternative, but it requires a university email address to get a free licence, and it seems to consume more memory than SDPA. To use Mosek, you can either use Picos and specify ``mosek7`` as the solver, or you can directly convert it to Mosek format from Ncpol2sdpa with the function ``convert_to_mosek``.

7.3 Execution on a cluster
-----------------------------------
SDPA is extremely well-suited to run on a cluster, or at least on a high-performance computing node. The procedure of submitting an SDP to a cluster can be greatly simplified. Follow step 1 in Section 7.1 to get a problem in SDPA format. Assume that you have a job script called ``job_sdp.sh`` on the cluster in your home directory that takes two parameters, the name of the SDPA file and the output file, and sends you an email when the calculation is ready. Then, if the hostname of the cluster is ``cluster``, you can use this script to upload the problem and schedule the execution:

```
#!/bin/bash
scp $1 cluster:
ssh username@cluster qsub job_sdp.sh $1 ${1%.*}.out
```
where the parameter is the name of the SDPA file (e.g., ``problem.dat-s``). You might have to change the command ``qsub`` to the job submission command used on the cluster.

References
======
[1] Pironio, S.; Navascués, M. & Acín, A. [Convergent relaxations of polynomial optimization problems with noncommuting variables](http://arxiv.org/abs/0903.4368). _SIAM Journal on Optimization_, 2010, 20, pp. 2157-2180.

[2] Wittek, P. [Ncpol2sdpa -- Sparse Semidefinite Programming Relaxations for Polynomial Optimization Problems of Noncommuting Variables](http://arxiv.org/abs/1308.6029). _To appear in the ACM Transactions on Mathematical Software_, 2015.